In [10]:
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
from IPython.display import Audio
import pandas as pd
import numpy as np 
import os
import soundfile as sf
import time
from joblib import Parallel, delayed
from functools import partial
from multiprocessing import Pool

In [2]:
#get all features out of giver file 
def get_features(file_path, file_name):
    y, sr = librosa.load(file_path)
    iter_ = int(y.shape[0]/sr)
    features = []
    for i in range(iter_):
        y_seg = y[i*sr:(sr*i+sr)]
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=y_seg, sr=sr))
        rms = np.mean(librosa.feature.rms(y=y_seg))
        spec_cent = np.mean(librosa.feature.spectral_centroid(y=y_seg, sr=sr))
        spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y_seg, sr=sr))
        rolloff = np.mean(librosa.feature.spectral_rolloff(y=y_seg, sr=sr))
        zcr = np.mean(librosa.feature.zero_crossing_rate(y_seg))
        mfcc = np.mean(librosa.feature.mfcc(y=y_seg, sr=sr, n_mfcc=20),axis=1)
        features.append([chroma_stft,rms,spec_cent,spec_bw,rolloff,zcr,*mfcc]) # starred expressions for mfcc to get 20 columns instead of 20 row
        # b = np.concatenate((b,a),axis=1) # ajout filename apres?
    
    return features
    

In [3]:
audio_dir='LA/'
# label = 'REAL'
folders = ['ASVspoof2019_LA_dev','ASVspoof2019_LA_eval','ASVspoof2019_LA_train']
sound_files_path = []
for folder in folders:
    files = os.listdir(os.path.join(audio_dir,folder,'flac'))
    for file in files:
        path = os.path.join(os.path.join(audio_dir,folder,'flac',file))
        if file.endswith('.flac'):
            sound_files_path.append(path)
len(sound_files_path) 
sound_files_path[0][28:41]

'LA_D_8848915.'

In [25]:
sound_files_path[0][28:40]

'LA_D_8848915'

In [20]:
#make dataframe with all the features
def make_features_df(audio_dir):
    feature_list = []
    for audio in audio_dir:
        filename = audio[28:40]
        file_features=get_features(audio,filename)
        for segment_features in file_features:
                feature_list.append(segment_features + [filename])

    features_df = pd.DataFrame(feature_list,columns = data_set.columns)
    # print(f'{filename} done')
    return features_df


In [36]:
n_proc = 8
data_set = pd.read_csv('Data/DATASET-balanced.csv')

start = time.time()
r = Parallel(n_jobs=n_proc)(delayed(make_features_df)([sound]) for sound in sound_files_path)
print('Time multiprocessing with joblib :', time.time() - start)

# asvs_features = make_features_df(sound_files_path)

/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/opt/conda/lib/python3.10/site-packages/librosa/core/pitch.py:101: Use

Time multiprocessing with joblib : 1987.7859189510345


In [29]:
169/10000*len(sound_files_path)/60

34.44755166666666

In [37]:
all_part = r

In [38]:
fulldf= pd.DataFrame(all_part[0])
for r in all_part[1:]:
    partialdf = pd.DataFrame(r)
    fulldf = pd.concat([fulldf,partialdf])
    

fulldf
    


,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,LABEL
0,0.581642,0.003070,2199.394712,1951.288198,4489.428156,0.101729,-465.421295,125.339874,-69.007561,70.611092,...,-8.866219,7.612215,-12.527738,9.243306,-17.393263,6.024668,-14.883923,-1.570087,-11.686265,LA_D_8848915
1,0.325574,0.093229,2345.454133,1599.777611,3984.376665,0.152854,-236.496674,100.673485,-32.507786,49.088181,...,-10.290752,-2.104480,-12.366513,-0.074342,-20.750038,-0.219439,-13.774378,-12.227905,-15.365830,LA_D_8848915
2,0.315612,0.107650,1792.799111,1426.987016,3165.870250,0.124190,-233.423737,128.119858,-33.989712,45.122196,...,-4.690394,-8.381293,-15.633381,-2.263502,-20.402868,-5.021841,-14.528839,-10.487343,-13.769796,LA_D_8848915
0,0.539373,0.001211,1397.669671,1798.662060,3206.245006,0.058805,-601.982971,157.632462,-29.607780,64.629547,...,-18.353567,-2.086941,-21.357689,1.418289,-19.528841,2.797719,-17.868378,-0.484152,-12.906013,LA_D_4477471
1,0.309486,0.177945,1369.867761,1431.153215,2583.005593,0.078857,-275.831085,104.976555,-20.991016,46.207661,...,-12.673950,-6.094771,-14.902205,0.231349,-26.601341,-16.148132,-9.064847,-9.427711,-17.687447,LA_D_4477471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,0.371958,0.108856,1874.572218,1644.345669,3483.484996,0.100042,-267.693481,112.351730,-38.577652,28.142616,...,-9.952971,-4.968857,-4.419859,-3.030780,-23.952742,2.336856,-18.854225,-8.214225,-4.492851,c/LA_T_51403
0,0.334515,0.126682,1887.510015,1628.934264,3496.209162,0.117687,-200.804459,128.001221,-38.918961,32.557312,...,-18.347965,1.964419,-15.538898,-1.297084,-12.130425,-5.399279,-15.976752,-13.512214,-8.315792,c/LA_T_83730
1,0.268613,0.169597,1588.532926,1506.019870,3034.958163,0.081332,-194.924606,117.109032,-29.384462,49.451450,...,-17.681974,0.498010,-7.854861,-2.573860,-16.794678,-10.481019,-14.802436,-13.050943,-10.861348,c/LA_T_83730
0,0.410026,0.138760,1677.571697,1590.618139,3213.096480,0.094649,-205.353165,128.130096,-34.798000,44.976448,...,-14.185932,-1.863424,-9.754604,-3.552697,-16.958033,-6.803551,-13.341378,-4.011154,-12.574131,c/LA_T_96703


In [39]:
fulldf.to_csv('Data/asvc_features.csv',index=False)